# 資料讀取與結構化

In [1]:
# 0~9代表屬性，其中Attribute[0] = Class 是我們要比較的對象
Attribute = [
    "Class",
    "age",
    "menopause",
    "tumor_size",
    "inv_nodes",
    "node_caps",
    "deg_malig",
    "breast",
    "breast_quad",
    "irradiat",
]

# 經整理過後的資料
patient_list = []

# 讀取檔案並整理格式
with open("breast-cancer.txt", "r") as file:
    for line in file:
        data = line.strip().split(",")
        (
            Class,
            age,
            menopause,
            tumor_size,
            inv_nodes,
            node_caps,
            deg_malig,
            breast,
            breast_quad,
            irradiat,
        ) = data
        next_patient_data = {
            "Class": Class,
            "age": age,
            "menopause": menopause,
            "tumor_size": tumor_size,
            "inv_nodes": inv_nodes,
            "node_caps": node_caps,
            "deg_malig": deg_malig,
            "breast": breast,
            "breast_quad": breast_quad,
            "irradiat": irradiat,
        }
        patient_list.append(next_patient_data)

In [2]:
# 測試格式化後的資料
print(patient_list[0])
print(patient_list[0].get(Attribute[1]))

{'Class': 'no-recurrence-events', 'age': '30-39', 'menopause': 'premeno', 'tumor_size': '30-34', 'inv_nodes': '0-2', 'node_caps': 'no', 'deg_malig': '3', 'breast': 'left', 'breast_quad': 'left_low', 'irradiat': 'no'}
30-39


# 計算屬性值機率與適合度

In [3]:
import math

def compute_prob(data):
    """
    計算各個屬性中，每一種可能值的出現機率
    用於計算 entropy

    參數:
        data: List[Dict[str, str]]
            已格式化的病人資料

    返回:
        prob: List[Dict[str, float]]
            列表內存放同屬性數量的字典，
            紀錄每個屬性中可能值的機率分布
    """
    num_of_attr = len(Attribute)

    # 存放各個屬性可能值的機率
    prob = []
    for _ in range(num_of_attr):
        prob.append({})
    
    # 計算每一種可能值的出現次數
    for d in data:
        for attr_idx in range(num_of_attr):
            if d.get(Attribute[attr_idx]) in prob[attr_idx].keys():
                prob[attr_idx][d.get(Attribute[attr_idx])] = prob[attr_idx][d.get(Attribute[attr_idx])] + 1
            else: 
                prob[attr_idx][d.get(Attribute[attr_idx])] = 1
    
    # 以總資料筆數求機率
    total = len(data)
    for attr_prob in prob:
        for key, value in attr_prob.items():
            attr_prob[key] = value / total
            
    return prob

def entropy(x, data):
    """
    計算屬性 x 的 entropy: H(x)
    用於計算 entropy 與 SU

    參數:
        x: str
            屬性名稱
        data: List[Dict[str, str]]
            已格式化的病人資料

    返回:
        float
            屬性 x 的entropy
    """
    # 找到屬性在 Attribute 中的索引
    attr_idx = Attribute.index(x)

    # 取得該屬性的機率分佈（dict: value -> prob）
    attr_prob = compute_prob(data)[attr_idx].values()

    # 計算 entropy
    return -sum(p * math.log2(p) for p in attr_prob)

def joint_entropy(x, y, data):
    """
    計算屬性 x, y 的 joint entropy: H(x, y)
    用於計算 SU

    參數:
        x, y: str
            屬性名稱
        data: List[Dict[str, str]]
            已格式化的病人資料

    返回:
        float
            屬性 x, y 的 joint entropy
    """
    # 計算所有 (x, y) 組合的出現次數
    pair_count = {}
    for d in data:
        pair = (d[x], d[y])
        if pair in pair_count:
            pair_count[pair] += 1
        else:
            pair_count[pair] = 1

    total = len(data)

    # 計算機率
    prob = [count / total for count in pair_count.values()]

    # 計算 joint entropy
    return -sum(p * math.log2(p) for p in prob)

#計算symmetric_uncertainty
def su(x, y, data):
    """
    計算 symmetric uncertainty
    用於計算 Goodness

    參數:
        x, y: str
            屬性名稱
        data: List[Dict[str, str]]
            已格式化的病人資料

    返回:
        float
            屬性 x, y 的 joint entropy
    """
    hx = entropy(x, data)
    hy = entropy(y, data)
    hxy = joint_entropy(x, y, data)

    denominator = hx + hy
    numerator = denominator - hxy

    # 避免分母為 0
    if denominator == 0:
        return 0.0
    
    return 2 * numerator / denominator

def goodness(subset, data):
    """
    計算 Goodness

    參數:
        subset: List[str]
            當前特徵子集包含的屬性
        data: List[Dict[str, str]]
            已格式化的病人資料

    返回:
        float
            特徵子集的 Goodness
    """

    # 計算分子
    numerator = 0.0
    target = Attribute[0]   # Class

    for feature in subset:
        numerator += su(feature, target, data)
    
    # 計算分母
    if len(subset) == 1:
        denominator = 1.0
    else:
        denominator = 0.0
        for f1 in subset:
            for f2 in subset:
                denominator += su(f1, f2, data)
    
    # 避免分母為 0
    if denominator == 0:
        return 0.0
    
    return numerator / math.sqrt(denominator)


In [4]:
# 測試計算得來的機率
print(compute_prob(patient_list))

# 測試 H(x)
for i in range(3):
    print(f"entropy: {entropy(Attribute[i], patient_list)}")

# 測試 H(x, y)
print(f"joint entropy: {joint_entropy(Attribute[0], Attribute[1], patient_list)}")

# 測試 SU
print(f"SU: {su(Attribute[0], Attribute[1], patient_list)}")

# 測試 Goodness
print(f"goodness: {goodness(['inv_nodes'],patient_list)}")

[{'no-recurrence-events': 0.7075812274368231, 'recurrence-events': 0.2924187725631769}, {'30-39': 0.1299638989169675, '40-49': 0.3212996389891697, '60-69': 0.19855595667870035, '50-59': 0.3285198555956679, '70-79': 0.018050541516245487, '20-29': 0.0036101083032490976}, {'premeno': 0.5379061371841155, 'ge40': 0.44404332129963897, 'lt40': 0.018050541516245487}, {'30-34': 0.20577617328519857, '20-24': 0.17328519855595667, '15-19': 0.10469314079422383, '0-4': 0.02888086642599278, '25-29': 0.18411552346570398, '50-54': 0.02888086642599278, '10-14': 0.10108303249097472, '40-44': 0.07942238267148015, '35-39': 0.06859205776173286, '5-9': 0.01444043321299639, '45-49': 0.010830324909747292}, {'0-2': 0.7545126353790613, '6-8': 0.061371841155234655, '9-11': 0.02527075812274368, '3-5': 0.12274368231046931, '15-17': 0.021660649819494584, '12-14': 0.010830324909747292, '24-26': 0.0036101083032490976}, {'no': 0.7978339350180506, 'yes': 0.20216606498194944}, {'3': 0.296028880866426, '2': 0.465703971119

# Forward & Backward Selection 主程式

In [5]:
# Forward Selection

print("===== Forward Selection=====")

f_feature_subset = []
f_remaining_attributes = ['age', 'menopause', 'tumor_size', 'inv_nodes',
             'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
f_best_goodness = 0.0
f_iteration = 0

while True:

    best_goodness_in_cycle = 0.0
    f_iteration = f_iteration + 1
    
    # 每次在子集裡尋找一個特徵，使得將其加入子集後 Goodness 是最好
    for attr_idx in range(len(f_remaining_attributes)):
        new_feature_subset = f_feature_subset.copy()
        new_feature_subset.append(f_remaining_attributes[attr_idx])
        
#         print("選擇: ", f_remaining_attributes[attr_idx])
#         print("Gn: ", goodness(new_feature_subset, patient_list))
        
        if goodness(new_feature_subset, patient_list) > best_goodness_in_cycle:
            feature_to_add = f_remaining_attributes[attr_idx]
            best_goodness_in_cycle = goodness(new_feature_subset, patient_list)
            
    
    # 如果加入特徵不能再使 Goodness 增加，則停止
    if best_goodness_in_cycle < f_best_goodness:
        break
        
    
    # 如果可以，則更新子集和 Goodness
    f_feature_subset.append(feature_to_add)
    f_best_goodness = best_goodness_in_cycle
    f_remaining_attributes.remove(feature_to_add)
    
    print(f"第{f_iteration}次循環:")
    print(f"Selected Features: {f_feature_subset}")
    print(f"Best Goodness: {f_best_goodness:}")

print("Forward Selection result:", f_feature_subset,", Gn: ", f_best_goodness)

===== Forward Selection=====
第1次循環:
Selected Features: ['inv_nodes']
Best Goodness: 0.07664021945356746
第2次循環:
Selected Features: ['inv_nodes', 'deg_malig']
Best Goodness: 0.10145042616557834
第3次循環:
Selected Features: ['inv_nodes', 'deg_malig', 'node_caps']
Best Goodness: 0.10948734827994713
第4次循環:
Selected Features: ['inv_nodes', 'deg_malig', 'node_caps', 'irradiat']
Best Goodness: 0.11169589476594623
第5次循環:
Selected Features: ['inv_nodes', 'deg_malig', 'node_caps', 'irradiat', 'tumor_size']
Best Goodness: 0.11218633643681937
Forward Selection result: ['inv_nodes', 'deg_malig', 'node_caps', 'irradiat', 'tumor_size'] , Gn:  0.11218633643681937


In [6]:
# Backward Selection

print("===== Backward Selection=====")


b_feature_subset = ['age', 'menopause', 'tumor_size', 'inv_nodes',
             'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
b_best_goodness = goodness(b_feature_subset, patient_list)
b_iteration = 0


while True:

    best_goodness_in_cycle = 0.0
    b_iteration = b_iteration + 1
    
    # 每次在子集裡尋找一個特徵，使得將其剔除後 Goodness 是最好
    for attr_idx in range(len(b_feature_subset)):
        new_feature_subset = b_feature_subset.copy()
        new_feature_subset.remove(b_feature_subset[attr_idx])
        
#         print("選擇移除: ", b_feature_subset[attr_idx])
#         print("Gn: ", goodness(new_feature_subset, patient_list))
        
        # 檢查剔除該元素可否使 Goodness 增加，可則將欲元素和新的 Goodness 紀錄
        if goodness(new_feature_subset, patient_list) > best_goodness_in_cycle:
            feature_to_remove = b_feature_subset[attr_idx]
            best_goodness_in_cycle = goodness(new_feature_subset, patient_list)
            
    
    # 如果剔除屬性不能再使 Goodness 增加，則停止
    if best_goodness_in_cycle < b_best_goodness:
        break
        
    
    # 若可以，則更新子集與 Gn
    b_feature_subset.remove(feature_to_remove)
    b_best_goodness = best_goodness_in_cycle
    
    print(f"第{b_iteration}次循環:")
    print(f"Selected Features: {b_feature_subset}")
    print(f"Best Goodness: {b_best_goodness:}")

print("Backward Selection result:", b_feature_subset, ",Gn: ", b_best_goodness)

===== Backward Selection=====
第1次循環:
Selected Features: ['age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast_quad', 'irradiat']
Best Goodness: 0.09719419884125785
第2次循環:
Selected Features: ['age', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast_quad', 'irradiat']
Best Goodness: 0.10245371563812918
第3次循環:
Selected Features: ['age', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'irradiat']
Best Goodness: 0.10821070155745492
第4次循環:
Selected Features: ['tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'irradiat']
Best Goodness: 0.11218633643681936
Backward Selection result: ['tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'irradiat'] ,Gn:  0.11218633643681936
